In [1]:
# ### Step 1: Imports & Logging Setup
import os
import pandas as pd
import logging
from datetime import datetime

# Create logs folder if it doesn’t exist
if not os.path.exists("../logs"):
    os.makedirs("../logs")

# Logging configuration
logging.basicConfig(
    filename="../logs/ingestion.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger()


In [2]:
# ### Step 2: Load the raw dataset
data_path = r"C:\Users\pro\Downloads\medicare-drug-spending-analysis\data\raw\medicare_partD_drug_spending.csv"

try:
    df = pd.read_csv(data_path)
    logger.info(f"Dataset loaded successfully from {data_path} with shape {df.shape}")
    print(f"Dataset loaded: {df.shape}")
except FileNotFoundError:
    logger.error(f"File not found at {data_path}")
    raise


Dataset loaded: (14309, 46)


In [3]:
# ### Step 3: Preview data
print(df.head())
logger.info(f"Columns: {list(df.columns)}")


                       Brnd_Name             Gnrc_Name  Tot_Mftr  \
0       1st Tier Unifine Pentips  Pen Needle, Diabetic         1   
1       1st Tier Unifine Pentips  Pen Needle, Diabetic         1   
2  1st Tier Unifine Pentips Plus  Pen Needle, Diabetic         1   
3  1st Tier Unifine Pentips Plus  Pen Needle, Diabetic         1   
4                       Abacavir      Abacavir Sulfate         5   

         Mftr_Name  Tot_Spndng_2019  Tot_Dsg_Unts_2019  Tot_Clms_2019  \
0          Overall        139201.68           642471.0         5392.0   
1  Owen Mumford Us        139201.68           642471.0         5392.0   
2          Overall        343031.42          1830596.0        14581.0   
3  Owen Mumford Us        343031.42          1830596.0        14581.0   
4          Overall      10110328.45          3316293.0        42629.0   

   Tot_Benes_2019  Avg_Spnd_Per_Dsg_Unt_Wghtd_2019  Avg_Spnd_Per_Clm_2019  \
0          1878.0                         0.216788              25.816335  

In [4]:
# ### Step 4: Save a processed copy
processed_path = r"C:\Users\pro\Downloads\medicare-drug-spending-analysis\data\processed"
if not os.path.exists(processed_path):
    os.makedirs(processed_path)

output_file = os.path.join(processed_path, "medicare_drug_spending_clean.csv")
df.to_csv(output_file, index=False)
logger.info(f"Saved processed dataset to {output_file}")
print(f"Processed dataset saved to {output_file}")


Processed dataset saved to C:\Users\pro\Downloads\medicare-drug-spending-analysis\data\processed\medicare_drug_spending_clean.csv


In [5]:
 
logger.info(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

Rows: 14309, Columns: 46


In [6]:
duplicates = df.duplicated().sum()
logger.info(f"Duplicate rows: {duplicates}")
print(f"Duplicate rows: {duplicates}")

Duplicate rows: 0


In [7]:
missing = df.isnull().sum()
logger.info(f"Missing values per column:\n{missing}")
print(missing)


Brnd_Name                             0
Gnrc_Name                             0
Tot_Mftr                              0
Mftr_Name                             0
Tot_Spndng_2019                    4205
Tot_Dsg_Unts_2019                  4205
Tot_Clms_2019                      4205
Tot_Benes_2019                     4292
Avg_Spnd_Per_Dsg_Unt_Wghtd_2019    4205
Avg_Spnd_Per_Clm_2019              4205
Avg_Spnd_Per_Bene_2019             4399
Outlier_Flag_2019                  4207
Tot_Spndng_2020                    3258
Tot_Dsg_Unts_2020                  3258
Tot_Clms_2020                      3258
Tot_Benes_2020                     3435
Avg_Spnd_Per_Dsg_Unt_Wghtd_2020    3258
Avg_Spnd_Per_Clm_2020              3258
Avg_Spnd_Per_Bene_2020             3518
Outlier_Flag_2020                  3262
Tot_Spndng_2021                    2278
Tot_Dsg_Unts_2021                  2278
Tot_Clms_2021                      2278
Tot_Benes_2021                     2590
Avg_Spnd_Per_Dsg_Unt_Wghtd_2021    2278


In [8]:
logger.info(f"Data types:\n{df.dtypes}")
print(df.dtypes)

Brnd_Name                           object
Gnrc_Name                           object
Tot_Mftr                             int64
Mftr_Name                           object
Tot_Spndng_2019                    float64
Tot_Dsg_Unts_2019                  float64
Tot_Clms_2019                      float64
Tot_Benes_2019                     float64
Avg_Spnd_Per_Dsg_Unt_Wghtd_2019    float64
Avg_Spnd_Per_Clm_2019              float64
Avg_Spnd_Per_Bene_2019             float64
Outlier_Flag_2019                  float64
Tot_Spndng_2020                    float64
Tot_Dsg_Unts_2020                  float64
Tot_Clms_2020                      float64
Tot_Benes_2020                     float64
Avg_Spnd_Per_Dsg_Unt_Wghtd_2020    float64
Avg_Spnd_Per_Clm_2020              float64
Avg_Spnd_Per_Bene_2020             float64
Outlier_Flag_2020                  float64
Tot_Spndng_2021                    float64
Tot_Dsg_Unts_2021                  float64
Tot_Clms_2021                      float64
Tot_Benes_2

In [9]:
numeric_cols = [col for col in df.columns if 'Tot_Spndng' in col or 'Tot_Clms' in col]
negatives = (df[numeric_cols] < 0).sum()
logger.info(f"Negative values:\n{negatives}")
print(negatives)

Tot_Spndng_2019    0
Tot_Clms_2019      0
Tot_Spndng_2020    0
Tot_Clms_2020      0
Tot_Spndng_2021    0
Tot_Clms_2021      0
Tot_Spndng_2022    0
Tot_Clms_2022      0
Tot_Spndng_2023    0
Tot_Clms_2023      0
dtype: int64


In [10]:
range_check = df[numeric_cols].agg(['min', 'max'])
logger.info(f"Range check:\n{range_check}")
print(range_check)

     Tot_Spndng_2019  Tot_Clms_2019  Tot_Spndng_2020  Tot_Clms_2020  \
min     5.716000e+01           11.0     5.711000e+01           11.0   
max     7.304981e+09     50532357.0     9.936070e+09     55581508.0   

     Tot_Spndng_2021  Tot_Clms_2021  Tot_Spndng_2022  Tot_Clms_2022  \
min     2.643000e+01           11.0     3.306000e+01           11.0   
max     1.257515e+10     61088412.0     1.521981e+10     64797138.0   

     Tot_Spndng_2023  Tot_Clms_2023  
min     3.118000e+01             11  
max     1.827345e+10       68493698  


In [11]:
# df.isna().sum().sort_values(ascending=False).head(20)
df[df['Tot_Spndng_2019'].isna()]['Brnd_Name'].value_counts().head(10)


Brnd_Name
Lacosamide*                      21
Doxepin HCl*                     20
Fenofibrate                      18
Pantoprazole Sodium*             18
Potassium Chloride*              16
Levetiracetam*                   16
Emtricitabine-Tenofovir Disop    15
Triamcinolone Acetonide*         14
Lurasidone HCl                   14
Famotidine*                      14
Name: count, dtype: int64

In [12]:
# Convert Outlier_Flag to Boolean
outlier_cols = [col for col in df.columns if "Outlier_Flag" in col]
for col in outlier_cols:
    df[col] = df[col].apply(lambda x: True if x == 1 else False)

# Fill missing spending, claims, bene counts with 0 (assume no activity)
spending_cols = [col for col in df.columns if "Tot_Spndng" in col]
claims_cols = [col for col in df.columns if "Tot_Clms" in col]
bene_cols = [col for col in df.columns if "Tot_Benes" in col]
df[spending_cols + claims_cols + bene_cols] = df[spending_cols + claims_cols + bene_cols].fillna(0)

# Leave average spend columns as NaN


Outlier flags: CMS marked some drug-year combos as “weird.” We’ve converted that to True/False.

Spending/claims/beneficiaries blanks: We assume 0 activity (no sales/prescriptions).

Average spending blanks: We keep them blank

In [13]:
print(df[[col for col in df.columns if "Avg_Spnd" in col]].isna().sum())

Avg_Spnd_Per_Dsg_Unt_Wghtd_2019    4205
Avg_Spnd_Per_Clm_2019              4205
Avg_Spnd_Per_Bene_2019             4399
Avg_Spnd_Per_Dsg_Unt_Wghtd_2020    3258
Avg_Spnd_Per_Clm_2020              3258
Avg_Spnd_Per_Bene_2020             3518
Avg_Spnd_Per_Dsg_Unt_Wghtd_2021    2278
Avg_Spnd_Per_Clm_2021              2278
Avg_Spnd_Per_Bene_2021             2612
Avg_Spnd_Per_Dsg_Unt_Wghtd_2022    1326
Avg_Spnd_Per_Clm_2022              1326
Avg_Spnd_Per_Bene_2022             1735
Avg_Spnd_Per_Dsg_Unt_Wghtd_2023       0
Avg_Spnd_Per_Clm_2023                 0
Avg_Spnd_Per_Bene_2023              762
Chg_Avg_Spnd_Per_Dsg_Unt_22_23     1326
CAGR_Avg_Spnd_Per_Dsg_Unt_19_23    1161
dtype: int64


In [14]:
# Check for negative values in key columns
check_cols = [col for col in df.columns if "Tot_Spndng" in col or "Tot_Clms" in col]
negatives = (df[check_cols] < 0).sum()
print("Negative values:\n", negatives)




Negative values:
 Tot_Spndng_2019    0
Tot_Clms_2019      0
Tot_Spndng_2020    0
Tot_Clms_2020      0
Tot_Spndng_2021    0
Tot_Clms_2021      0
Tot_Spndng_2022    0
Tot_Clms_2022      0
Tot_Spndng_2023    0
Tot_Clms_2023      0
dtype: int64


In [15]:
# Check maximum spending per brand per year
for col in [c for c in df.columns if "Tot_Spndng" in c]:
    print(f"{col}: min={df[col].min()}, max={df[col].max()}")

Tot_Spndng_2019: min=0.0, max=7304981235.0
Tot_Spndng_2020: min=0.0, max=9936069814.0
Tot_Spndng_2021: min=0.0, max=12575145852.0
Tot_Spndng_2022: min=0.0, max=15219812294.0
Tot_Spndng_2023: min=31.18, max=18273451967.0


In [16]:
numeric_cols = [col for col in df.columns if "Tot_Spndng" in col or "Tot_Clms" in col]
outlier_summary = {}

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    outlier_summary[col] = len(outliers)

print("Outliers detected per column:\n", outlier_summary)


Outliers detected per column:
 {'Tot_Spndng_2019': 2372, 'Tot_Clms_2019': 2655, 'Tot_Spndng_2020': 2370, 'Tot_Clms_2020': 2571, 'Tot_Spndng_2021': 2293, 'Tot_Clms_2021': 2556, 'Tot_Spndng_2022': 2243, 'Tot_Clms_2022': 2536, 'Tot_Spndng_2023': 2236, 'Tot_Clms_2023': 2528}
